In [1]:
import plotly.express as px
import pandas as pd
from google.cloud import bigquery
import numpy as np
from datetime import time, timedelta, datetime
import math 
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ipywidgets
import statsmodels.stats.weightstats as sw
from statsmodels.stats.proportion import test_proportions_2indep
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.figure_format='retina'
from google.cloud import bigquery
from scipy.stats import norm
from scipy import stats
pd.options.display.float_format = "{:,.2f}".format
from scipy.stats import norm
from scipy import stats
import statsmodels.stats.weightstats as ws
import openpyxl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import AB_library
# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)
# df["Week"] = df["Date"].dt.to_period("W").dt.to_timestamp()

bigquery_client = bigquery.Client(project='analytics-dev-333113')

def cycle_sql(start, end, query, weeks=False):
    """
    You have to use {date} in your script to add cycle date into this backets
    """
    date_start = datetime.strptime(start, '%Y-%m-%d')
    date_end = datetime.strptime(end, '%Y-%m-%d')

    if weeks == False:
        daterange = [(date_start + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days)+1)]
    else:
        daterange = [(date_start + timedelta(weeks=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days//7)+1)] # weeks dividing days by 7

    total_df = pd.DataFrame()

    counter = 0

    for date in daterange:
        counter+=1
        print(f"{counter}) Uploading - {date}:", datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
        script = query.format(date = date)
        df_cycle = bigquery_client.query(script).to_dataframe()
        if df_cycle.empty == True:
            print('Dataframe is empty')
        total_df = pd.concat([df_cycle, total_df])
    return total_df

def read_bq(query, project='analytics-dev-333113'):
    client = bigquery.Client(project=project)
    query_job = client.query(query)
    result_df = query_job.to_dataframe()
    return result_df

def display_side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(
        html_str.replace('table','table style="display:inline"'), 
        raw=True
    )

def writing_excel(name:str, dataset1=None, dataset2=None, dataset3=None, dataset4=None):
    with pd.ExcelWriter(f"{name}.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet

        if dataset1 is not None:
            if dataset2 is not None:
                if dataset3 is not None:
                    if dataset4 is not None:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                        dataset4.to_excel(writer, sheet_name=f"4-{name}", 
                                        #   index=False
                                          )
                    else:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                else:
                    dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                    #   index=False
                                      )
                    dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                    #   index=False
                                      )
            else:
                dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                #   index=False
                                  )

        print('DataFrame is written to Excel File successfully.')

In [4]:
df = read_bq("""
WITH sharing AS (SELECT t1.event_dt_part,
                        IF((t3.mode = 'client'), 'pass', t3.mode) AS mode,
                        t2.city_name,
                        t2.city_id,
                        t2.country_name,
                        t2.country_id,
                        COUNT(t1.user_id)                         AS clicks,
                        COUNT(DISTINCT t1.user_id)                AS uniq_clicks
                 FROM indriver-e6e40.emart.swrve_event t1
                          JOIN indriver-e6e40.heap.vw_macroregion_mapping t2
                               ON
                                   t1.city_id = t2.city_id
                          JOIN dwh-storage-327422.personal_data.tbl_user_act t3
                               ON t1.user_id = t3.id
                 WHERE 1 = 1
                   AND t1.name IN (
                     'safety_center.share_ride_info.click'
                     )
                   AND t1.event_dt_part >= '2024-10-01'
                   AND t2.country_id IN (25, 11, 22, 10, 24, 60, 12)
                 GROUP BY 1, 2, 3, 4, 5, 6),
     rides AS (SELECT metric_date_utc, user_type, city_id, country_id, SUM(rides_count) AS rides
               FROM indriver-bi.incity.tbl_incity_growth_metrics_detail
               WHERE 1 = 1
               GROUP BY 1, 2, 3, 4)
SELECT t1.*,
       DATE_TRUNC(event_dt_part, WEEK)  AS weekly,
       DATE_TRUNC(event_dt_part, MONTH) AS monthly,
       t2.rides
FROM sharing t1
         JOIN rides t2
              ON t1.city_id = t2.city_id
                  AND t1.event_dt_part = t2.metric_date_utc
                  AND t1.mode = t2.user_type
""")

df.head()

,event_dt_part,mode,city_name,city_id,country_name,country_id,clicks,uniq_clicks,weekly,monthly,rides
0,2024-10-01,pass,Sheikhupura,5384,Pakistan,60,5,4,2024-09-29,2024-10-01,75
1,2024-10-01,pass,Passo Fundo,7165,Brazil,11,8,6,2024-09-29,2024-10-01,287
2,2024-10-01,driver,Chetumal,5261,Mexico,12,8,5,2024-09-29,2024-10-01,155
3,2024-10-01,driver,Rio do Sul,22331,Brazil,11,1,1,2024-09-29,2024-10-01,137
4,2024-10-01,driver,Mogi das Cruzes,25961,Brazil,11,6,5,2024-09-29,2024-10-01,20


In [18]:
df_agg_month = df.groupby(['mode', 'monthly', 'country_name'], as_index=False)[['clicks', 'uniq_clicks', 'rides']].sum()
df_agg_month['CTR_by_uniq_clicks'] = df_agg_month['uniq_clicks'] / df_agg_month['rides'] * 100
df_agg_month.pivot_table(columns='mode', index=['monthly', 'country_name'], values='CTR_by_uniq_clicks', aggfunc='max')

df_agg_week = df.groupby(['mode', 'weekly', 'country_name'], as_index=False)[['clicks', 'uniq_clicks', 'rides']].sum()
df_agg_week['CTR_by_uniq_clicks'] = df_agg_week['uniq_clicks'] / df_agg_week['rides'] * 100
df_agg_week['segment'] = df_agg_week['mode'] + ' - ' + df_agg_week['country_name']


In [42]:
df_agg_week[df_agg_week['country_name'].isin(['Pakistan', 'South Africa'])].groupby(['country_name'], as_index=False)[['clicks', 'uniq_clicks', 'rides']].mean().round(0)

,country_name,clicks,uniq_clicks,rides
0,Pakistan,"66,037.00","53,177.00","2,717,646.00"
1,South Africa,"18,362.00","14,201.00","554,324.00"


In [36]:
fig = px.line(
    df_agg_week, x="weekly", 
    y="CTR_by_uniq_clicks", 
    color='segment', 
    symbol="segment",
    width=1400, 
    height=700,
    title="CTR dynamics over the period. By Driver & Passenger types in most valuable countries"
    )

fig.show()